In [1]:
import cv2 as cv
import numpy as np


In [2]:
sunglasses = cv.imread('C:/Users/koosha/Desktop/sunglasses_with_alpha.png',cv.IMREAD_UNCHANGED)
face = cv.imread('download1.jpg',cv.IMREAD_UNCHANGED)

In [3]:
# بارگذاری مدل تشخیص چشم
eyes_cascade = cv.CascadeClassifier('frontalEyes.xml')

# شروع وبکم
cap = cv.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    face = frame.copy()
    gray = cv.cvtColor(face, cv.COLOR_BGR2GRAY)

    # تشخیص چشم‌ها
    eyes = eyes_cascade.detectMultiScale(gray, 1.3, 5)

    if len(eyes) >= 2:
        eyes = sorted(eyes, key=lambda e: e[0])  
        (x1, y1, w1, h1) = eyes[0]
        (x2, y2, w2, h2) = eyes[1]

        x = min(x1, x2)
        y = min(y1, y2)
        x2_max = max(x1 + w1, x2 + w2)
        y2_max = max(y1 + h1, y2 + h2)

        w = x2_max - x
        h = int(1.5 * (y2_max - y))

        sung_resized = cv.resize(sunglasses, (w, h), interpolation=cv.INTER_AREA)

        if y + h < face.shape[0] and x + w < face.shape[1]:  
            roi = face[y:y+h, x:x+w]

            if sung_resized.shape[2] == 4:
                alpha_s = sung_resized[:, :, 3] / 255.0
                alpha_f = 1.0 - alpha_s

                for c in range(3):  # BGR
                    roi[:, :, c] = (alpha_s * sung_resized[:, :, c] +
                                    alpha_f * roi[:, :, c])

                face[y:y+h, x:x+w] = roi

    cv.imshow('Glasses Filter - Live', face)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
